### Get all flight codes from OpenFlight website
https://openflights.org/data.php#license

##### Import

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv('./../secrets.env')

In [2]:
raw_airport_data_path = "raw_airports.csv"

##### Read file

In [3]:
df = pd.read_csv(raw_airport_data_path, header=None)

##### Rename Column

In [4]:
rename_columns = {
    0: 'airport_id', 
    1: 'name', 
    2: 'city', 
    3: 'country', 
    4: 'IATA', 
    5: 'ICAO', 
    6: 'latitude', 
    7: 'longitude', 
    8: 'altitude', 
    9: 'timezone', 
    10: 'dst', 
    11: 'tz_database',
    12: 'type', 
    13: 'source'}

In [5]:
df = df.rename(columns=rename_columns)

##### Replace na values to None

In [6]:
df_replaced_none = df.replace('\\N', None)

In [ ]:
df_replaced_none

In [ ]:
df_replaced_none.info()

##### Get relevant columns

In [9]:
df_relevant_columns = df_replaced_none[['name', 'city', 'country', 'IATA']]

##### Drop empty IATA rows and empty city rows

In [10]:
# mask = (df_relevant_columns['IATA'].isna()) | (df_relevant_columns['city'].isna())
mask = (df_relevant_columns['IATA'].isna())

In [11]:
df_relevant_columns_nona = df_relevant_columns[~mask]

In [ ]:
df_relevant_columns_nona

In [13]:
df_relevant_columns_nona = df_relevant_columns_nona.rename(columns={'name': 'airport_name', 'IATA': 'iata'})

In [ ]:
df_relevant_columns_nona.describe()

In [20]:
df_relevant_columns_nona['city'] = df_relevant_columns_nona['city'].fillna('')

#### Put dataframe to Mysql

In [ ]:
pip install pymysql sqlalchemy

In [22]:
sql_query = """
    SELECT `airport_name`, `city`, `iata` FROM `airportcodes` WHERE `iata` LIKE %s OR `city` LIKE %s OR `airport_name` LIKE %s ORDER BY CASE WHEN `iata` = %s THEN 1 WHEN `iata` LIKE %s THEN 2 WHEN `city` LIKE %s THEN CASE WHEN `city` LIKE %s THEN 3 WHEN `city` LIKE %s THEN 4 ELSE 5 END WHEN `airport_name` LIKE %s THEN CASE WHEN `airport_name` LIKE %s THEN 6 WHEN `airport_name` LIKE %s THEN 7 ELSE 8 END ELSE 9 END LIMIT 5
"""

sql_query = """
                SELECT `airport_name`, `city`, `iata`
                FROM `airportcodes`
                WHERE `iata` LIKE %s
                OR `city` LIKE %s
                OR `airport_name` LIKE %s
                ORDER BY
                    CASE
                        -- Exact match for `iata` has the highest priority
                        WHEN `iata` = %s THEN 1
                        -- Matches where the target word is at the beginning or end of the string
                        WHEN `iata` LIKE %s THEN 2
                        WHEN `city` LIKE %s THEN
                            CASE
                                WHEN `city` LIKE %s THEN 3  -- Starts with the target word
                                WHEN `city` LIKE %s THEN 4  -- Ends with the target word
                                ELSE 5
                            END
                        WHEN `airport_name` LIKE %s THEN
                            CASE
                                WHEN `airport_name` LIKE %s THEN 6  -- Starts with the target word
                                WHEN `airport_name` LIKE %s THEN 7  -- Ends with the target word
                                ELSE 8
                            END
                        ELSE 9
                    END
                LIMIT 5
            """

In [ ]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine


db_data = 'mysql+pymysql://' + os.getenv('MYSQL_USERNAME') + ':' + os.getenv('MYSQL_PASSWORD') + '@' + 'localhost' + os.getenv('MYSQL_PORT') + os.getenv('MYSQL_DB')
engine = create_engine(db_data)

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user=os.getenv('MYSQL_USERNAME'),
                         password=os.getenv('MYSQL_PASSWORD'),
                         db=os.getenv('MYSQL_DB'))    

# create cursor
cursor=connection.cursor()
# Execute the to_sql for writing DF into SQL !!IMPORTANT!!!
df_relevant_columns_nona.to_sql(os.getenv('MYSQL_DB'), engine, if_exists='replace', index=False)    

# Execute query
# sql_query = "SELECT `airport_name`, `city`, `iata` FROM `airportcodes` WHERE `airport_name` LIKE %s OR `city` LIKE %s OR `iata` LIKE %s ORDER BY CASE WHEN `iata` = %s THEN 1 WHEN `city` LIKE %s THEN 2 WHEN `airport_name` LIKE %s THEN 3 ELSE 4 END LIMIT 5"
query = 'man'
search_term = f'%{query}%'
starts_with = f'{query}%'
ends_with = f'%{query}'

cursor.execute(sql_query, (
                query, search_term, search_term,
                query, search_term, search_term,
                starts_with, ends_with, search_term,
                starts_with, ends_with
            ))

# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

engine.dispose()
connection.close()

##### Save file to data folder

In [ ]:
airport_code_path = os.path.dirname(os.path.dirname(os.getcwd())) + '\\data\\'

In [ ]:
airport_code_path

In [ ]:
df_relevant_columns_nona.to_csv(airport_code_path + 'all_airport_code.csv', index=False)